In [1]:
import openai
import csv
import os
import openai
import pandas as pd
import collections
import ast
import time
import collections
import json
import sqlite3
import tiktoken
import xxhash
import numpy as np
from openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

In [2]:
# fill in the openai api key
os.environ["OPENAI_API_KEY"] = ""

openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
import pickle 
def save(fname, d):
    with open(fname, 'wb') as f:
        pickle.dump(d, f)


In [5]:
import time as time
from multiprocessing import Process, Queue
import query_module

# def handler(signum, frame):
#     print('time out')
#     raise TimeoutError("Query execution exceeded the allowed time limit.")


def evalfunc(sql_source, sql_target, database, source='kaggle'):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
    if not os.path.isfile(db_path):
        print("cannot find file", db_path)
        return False
    timeout = 120
    output = Queue()
    query_process = Process(target=query_module.execute_query, args=(db_path, sql_source, output))
    query_process.start()
    output_hash = ''
    
    try:
        # Connect to sqlite db
        # Execute both!
        source_results = None
        source_results = output.get(True, timeout+5)
        query_process.join(timeout)
        if query_process.is_alive():
            print("process terminated")
            query_process.terminate()  # Terminate the process
            query_process.join()  # Make sure it's cleaned up
            return False, [Exception('SQL query took too much time to execute.')]
        if isinstance(source_results, Exception):
            raise source_results
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        target_results = cursor.execute(sql_target).fetchall()
        cursor.close()
        connection.close()
        # If the lengths don't match... there's no hope
        if len(source_results) != len(target_results):
            # (result matches or not, valid, hash)
            return False, []
        if 'ORDER BY' in sql_target:
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                rhs = tuple(sorted(list(b), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
                if lhs != rhs:
                    # Oh no, a row doesn't match!
                    return False, []
        else:
            lset, rset = set(), set()
            for a, b in zip(source_results, target_results):
                # NOTE: we are doing compares that are column-order independent
                # hence the sorting and the weird key (since we may have mixed
                # types in a row)
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
                rset.add(tuple(sorted(list(b), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
            if lset != rset:
                # Oh no, rows don't match!
                return False, []
    # If we hit an error, that's not a match I guess...
    except Exception as ex:
        print(ex)
        return False, [ex]
    return True, []


def outputHash(sql_source, database):
    db_path = f'./databases/{database}/{database}.sqlite'
    output_hash = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        source_results = cursor.execute(sql_source).fetchall()
        output_hash = xxhash.xxh128_hexdigest(str(len(source_results)), seed=123)
        if 'ORDER BY' in sql_source:
            for a in source_results:
                lhs = tuple(sorted(list(a), key=lambda x: hash(x)))
                output_hash = xxhash.xxh128_hexdigest(output_hash + str(lhs), seed=123)
        else:
            lset = set()
            for a in source_results:
                lset.add(tuple(sorted(list(a), key=lambda x: hash(x))))
            output_hash = xxhash.xxh128_hexdigest(str(lset), seed=123)
    except Exception as ex:
        return False
    finally:
        cursor.close()
        connection.close()
    return output_hash


def execute(sql, database, source):
    assert source in ['kaggle', 'bird']
    db_path = ''
    if source == 'kaggle':
        db_path = f'./databases/{database}/{database}.sqlite'
    elif source == 'bird':
        db_path = f'./bird_dev/dev_databases/{database}/{database}.sqlite'
        
    if not os.path.isfile(db_path):
        print("cannot find file")
        return False
    results = ''
    try:
        # Connect to sqlite db
        connection = sqlite3.connect(db_path)
        cursor = connection.cursor()
        results = cursor.execute(sql).fetchall()
    # If we hit an error, that's not a match I guess...
    except KeyboardInterrupt:
        cursor.close()
        connection.close()
        print("KeyboardInterrupt")
        return False
    except Exception as ex:
        cursor.close()
        connection.close()
        print(ex)
        return False
    finally:
        cursor.close()
        connection.close()
    return results

In [6]:
def GPT4_turbo_generation(prompt, t = 0.0, p = 0.0, stopWord="Answer:",):
    response = client.chat.completions.create(
        model = 'gpt-4-turbo-preview',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4000,
        top_p = p,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        logprobs=True,
        stop = [stopWord]
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

def GPT35_generation(prompt, t = 0.0, p=0.0, fp=0.0, pp=0.0, stopWord="Answer:",):
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages=[{"role": "user", "content": prompt}],
        n = 1,
        stream = False,
        temperature=t,
        max_tokens=4000,
        top_p = p,
        frequency_penalty=fp,
        presence_penalty=pp,
        logprobs=True,
        stop = [stopWord]
    )
    logprobs = [token.logprob for token in response.choices[0].logprobs.content]
    perplexity_score = np.exp(-np.mean(logprobs))
    return response.choices[0].message.content.strip(), perplexity_score

In [7]:
df = pd.read_csv('kaggle_dataset.csv')

In [8]:
bird_data_df = pd.read_json('sampled_bird_dev.json')
bird_data_df.reset_index(level=0, inplace=True)

In [9]:
userstudy = []
with open('./logs/user_study.pkl', 'rb') as f:
    userstudy = pickle.load(f)

In [10]:
survey_questions = []
# add original gold query inside
for d in userstudy:
    if 'Question2Ask' in d:
        assert len(d['Question2Ask']) == len(d['Answer2Question']), print(d['nl'])
    q = d['nl']
    sql = df.loc[df['nl'] == q]['sql'].values
    d['gold'] = sql[0]
    d["target_schema"] = df.loc[df['nl'] == q]['target_schema'].values
    survey_questions.append(q)

print(len(df))
# drop the user study questions
df = df[~df['nl'].isin(survey_questions)]
print(len(df))
    

272
208


In [11]:
df = df.reset_index()

In [12]:
def clean_query(sql_query):
    sql_query = sql_query.replace("```sql", '')
    sql_query = sql_query.replace("`", '')
    sql_query = sql_query.replace(';', '')
    sql_query = sql_query.replace('"""', '')
    return sql_query


In [13]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [14]:
feedback_prefix_v1='''Example Natural Language Question:```'How many acres burned in fires in California each year between 2000 and 2005?'```
Example Gold Query that answers Natural Language Question:```'SELECT\n  SUM(FIRE_SIZE),\n  FIRE_YEAR\nFROM Fires\nWHERE\n  State = "CA" AND FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  FIRE_YEAR'```
Example Clarification Question:```What information should the output table contain? a) output table has two columns: the total acres burned and the year, b) output table has one column: the total acres burned for each year, c) output table has one column: the total acres burned across all target years, d) other (please specify).```
Example Reasoning:```The clarification question is asking what information should the output table contain. The fields in the output table are determined by the SELECT clause in the gold query. The gold query uses ‘SELECT  SUM(FIRE_SIZE), FIRE_YEAR’ to select the outputs. As a result, the output table has two columns, the total acres burned and the year. Hence, choice a is correct.```
Example Answer:```a) output table has two columns: the total acres burned and the year```

Example Natural Language Question:```'Which states had the largest number of fires in 2001?’```
Example Gold Query that answers Natural Language Question:```SELECT\n  State\nFROM Fires\nWHERE\n  FIRE_YEAR = 2001\nGROUP BY\n  State\nORDER BY\n  COUNT(*) DESC\nLIMIT 1```
Example Clarification Question:```Is the largest number of fires referring to? a) the total size of all fire incidents, b) the number of fire incidents, c) the largest size of all fire incidents, d) other (please specify).```
Example Reasoning:```The clarification question is asking about how to represent the largest number of fires. The gold query uses ‘ORDER BY COUNT(*) DESC LIMIT 1’ to find the largest number of fires. As a result, choice a is correct.```
Example Answer:```b) the number of fire incidents```

Example Natural Language Question:```What was the most common cause of fire between 2000 and 2005?```
Example Gold Query that answers Natural Language Question:```'SELECT\n  STAT_CAUSE_DESCR\nFROM Fires\nWHERE\n  FIRE_YEAR BETWEEN 2000 AND 2005\nGROUP BY\n  STAT_CAUSE_DESCR\nORDER BY\n  COUNT(*) DESC\nLIMIT 1'```
Example Clarification Question:```Which information should be used to represent the 'cause of fire'? a) the code that represents the cause, b) the description of the cause, c) both the code and the description of the cause, d) other (please specify).```
Example Reasoning:```The clarification question is asking for which column should be used to represent the cause of fire. The gold query uses the STAT_CAUSE_DESCR to represent the cause. STAT_CAUSE_DESCR column contains information about the description of the causes. As a result, choice b is correct.```
Example Answer:```b) the description of the cause```

Example Natural Language Question:```What was the cause of the largest wildfire in Utah in the year 1997?```
Example Gold Query that answers Natural Language Question:```SELECT\n  *\nFROM Fires\nWHERE\n  State = "UT" AND FIRE_YEAR = 1997\nORDER BY\n  FIRE_SIZE DESC\nLIMIT 1```
Example Clarification Question:```Is the cause of fire referring to? a) the code that represents the cause, b) the description of the cause, c) both the code and the description of the cause, d) other (please specify).'```
Example Reasoning:```The clarification question is asking for what columns represent the cause of fire. The natural language question is asking for the causes and the gold query is selecting all fields in the database to the output table. Hence, based on the gold query, the cause of the fire is represented by all fields from the Fires table. As a result, the correct answer is d, since a, b and c are all wrong.```
Example Answer:```d) other (please specify). Please use all fields in the fires table to represent the 'cause' information.```

Example Natural Language Question:```Whose CDs sells best?```
Example Gold Query that answers Natural Language Question:```'SELECT\n  artist\nFROM torrents\nGROUP BY\n  artist\nORDER BY\n  SUM(totalSnatched) DESC\nLIMIT 1'```
Example Clarification Question:```Which column should be used to identify music related to 'CD'? a) groupName, b) tag, c) releaseType, d) other (please specify)```
Example Reasoning:```The clarification question is asking for what columns represent the CDs. The gold query does not use a WHERE clause to filter the CDs. Hence, the CD information is not contained in the tag column or the release type column. As a result, choice a, b, and c are all wrong.```
Example Answer:```d) other (please specify). Consider all music. No 'CD' filters on group name or tag or release type.```
'''

feedback_v2 = """Your task is to answer the multiple choice clarification question truthfully based on the Gold Query.

Natural Language Question: ```{nlq}```
Gold Query: ```{query}```
The gold query is the sql answer to the natural language question

Multiple Choice Clarification Question: ```{question}```

Answer the above clarification question truthfully based on the Gold Query.

First, reason which portion of the gold query answers the clarification question.
Next, consider the correctness of each multiple choice answers based only on the gold query.
Lastly, output the answer with format: answer_to_cq = "".
If none of the multiple choices are correct or you selected `other (please specify)`, give a short answer to the clarification question after `other (please specify)`.


Let's think step by step:
"""



In [15]:
cq_prefix_v1 = '''/* some examples are provided before the main question */\n\n
Example Question: ```Which artist/group is most productive?```
Example Reasoning:```Based on the question 'Which artist/group is most productive?', it is clear that the SQL answer should contain the 'most productive artist/group’. Hence, the SQL answer needs to contain ORDER BY and LIMIT 1. However, the question itself is ambiguous, and how productivity is determined needs to be clarified. Sorting based on the number of music produced or the sum of total downloads are both valid choices. In addition, it is unclear what columns should be used to represent the 'artist/group'. The artist and the groupName columns contain information about 'artist/group'.```
Example Ambiguity Type:```[‘AmbQuestion’, ‘AmbColumn’]```
Example Multiple Choice Clarification Question:```How to rank artist/group productivity? a) rank by the number of records produced, b) rank by the total number of downloads, c) other (please specify).
Which columns represent the 'artist/group' information? a) the artist column only, b) the groupName column only, c) both the artist column and the groupName column, d) other (please specify).```

Example Question: ```Which Premier League matches ended in a draw in 2016?```
Example Reasoning:```It is clear that the SQL answer to this question needs to contain a WHERE clause for three conditions: 'Premier League', 'draw', and 'in 2016'. However, it needs to be clarified i) which column should be used in the WHERE clause and ii) what the predicate value should be used. For instance, to find 2016, it is unclear if the SQL answer should use the season column or the DateTime column. Also, there are several choices for the predicate value, such as '2016', '2016/2017', or '2015/2016'. Moreover, the output table is unclear. The question did not specify what fields should be contained in the output table.```
Example Ambiguity Type:```['AmbColumn', 'AmbValue', 'AmbOutput']```
Example Multiple Choice Clarification Question:```Is the year '2016' referring to? a) season is 2016, b) season is either 2015/2016 or 2016/2017, c) the date time is at year 2016, d) other (specify).
How to find the 'Premier league'? a) consider all leagues, b) consider only the league with name 'Premier League', c) consider only the league that contains the name ‘Premier’, d) other (specify).
What fields represent the target 'matches'? a) all fields from football data table, b) a single column with league information, c) other (specify).```

Example Question: ```Which type of crime has the highest rate of ‘Investigation complete’?```
Example Reasoning:```It is clear that the SQL answer to this question needs to contain a WHERE clause to find crimes that have 'Investigation complete' outcomes, uses ORDER BY and LIMIT 1 to find the type of crime with the highest rate, and the output table has only one row. However, it needs to be clarified i) what predicate value should be used for 'Investigation complete', and ii) how to represent the 'rate', and iii) if the output table contains only the crime type column or the crime type column with the highest rate aggregate.```
Example Ambiguity Type:```['AmbQuestion', 'AmbVal', ‘AmbOutput’]```
Example Multiple Choice Clarification Question:```What information should be used to find 'Investigation complete'? a) see if outcome contains the phrase 'Investigation complete', b)  see if outcome is 'Investigation complete; no suspect identified', c) other (please specify).
How to represent the 'rate' for each crime type? a) the number of crimes for each crime type over all crimes, b) a) the number of crimes for each crime type over the all crimes that are 'Investigation Complete', c) the number of crimes for each crime type, c) other (please specify).
What fields should the output table contain? a) two fields: the crime type and the rate, b) one field: the crime type only, c) other (please specify).```

Example Question: ```For award winners, which position that has the most hall of fame players?```
Example Reasoning:```It is clear that the 'award winners' information is contained in the player_award table, the 'hall of fame players' are players whose induction is 'Y' in the hall_of_fame table, and the output table only has a single row. However, it is unclear which column contains the position information. In addition, the question needs to clarify if the output should contain only the position information or the position information along with the number of hall-of-fame players.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbOutput’]```
Example Multiple Choice Clarification Question:```How should the 'position' for players be identified? a) by the 'award_id' column, b) by the 'category' column, c) by the 'notes' column, d) other (please specify).
What fields should be contained in the output table? a) one field: the position, b) two fields: the position and the number of hall-of-fame players, c) other (please specify).```

Example Question: ```How many Wisconsin school districts receive federal funding?```
Example Reasoning:```The output should contain only the number of Wisconsin school districts that receive federal funding. However, it is unclear which column and predicate values should be used to find 'Wisconsin school districts' and how to determine if a district has 'received federal funding'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Is 'Wisconsin school districts' referring to? a) all school districts in the state Wisconsin, b) school districts with names that contain Wisconsin, c) other (please specify).
How should the answer determine if a district has received federal funding? a) based on the t_fed_rev is larger than 0, b) the answer does not need to consider this aspect, c) other (please specify).```

Example Question: ```How many students from the ages of 5 to 17 are enrolled at the State Special School school in Fremont for the 2014-2015 academic year?```
Example Reasoning:```It is clear that the phrase 'students from the ages of 5 to 17 are enrolled
' from the question refers to the `Enrollment (Ages 5-17)` column in the frpm table, and the phrase '2014-2015 academic year' filters the result based on `Academic Year` column in the frpm table. However, it needs to be clarified which columns should be used to find results related to the 'State Special School school' and 'Fremont', and what predicate values should be used to represent 'State Special School school' and 'Fremont'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Which column contains the information about 'State Special School'? a) `School Type` in the frpm table, b) `EdOpsCode` in the schools table, c) `EILCode` in the schools table, d) other (please specify).
Which column contains the information about 'Fremont'? a) `City` column in the schools table, b) `County Name` in the frpm table, c) `District Name`  name in the frpm table, d) other (please specify).
What predicate value should be used to represent 'State Special School'? a) an exact string match on 'SSS', b) any name like '%State Special%', c) an exact string match on 'State Special School', d) other (please specify).```

Example Question: ```What is the average number of crimes committed in 1995 in regions where the number exceeds 4000 and the region has accounts that are opened starting from the year 1997?```
Example Reasoning:```It is clear that the output table contains the average number of crimes committed in 1995, and there needs to be a where clause to filter the year based on '1997' and filter the number of crimes committed in 1995 with '4000'. However, it needs to be clarified which columns represent the number of crimes committed in 1995 and how to find years after '1997'.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```
Which column contains the information about the 'number of crimes committed in 1995'? a) `A15` column in the district table, b) `A5` column in the district table, c) other (please specify).
How do I filter the account based on the year 1997? a) extract the year from the `date` column in the account table and then find the year larger or equal to 1997, b) similar to a) except use the trans
instead of account table, c) similar to a) except use the loan table instead of account table, d) other (please specify).```

Example Question: ```How many molecules have a double bond type and among these molecule, how many are labeled as carcinogenic compounds?```
Example Reasoning:```The output table clearly has two columns: the number of unique molecules and the number of carcinogenic compounds. The results should be filtered based on the 'double bound type' and 'carcinogenic compound'. However, it needs to be clarified on which column and what predicate value represents the 'double bond type' and 'carcinogenic compound'. ```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’]```
Example Multiple Choice Clarification Question:```How to represent the double bound type? a) double bound type is identified based on the `bond_type` column in the bond table using predicate value '='
, b) double bound type is identified based on the `bond_type` column in the bond table using predicate value 'double', c) other (please specify)
How to find 'carcinogenic compound'? a) it can be identified using the `label` column in the molecule table with predicate value '+', b) it can be identified using the `label` column in the molecule table with predicate value '++', c) other (please specify)```
Example Question: ```What is the status of card "Cloudchaser Eagle"?```
Example Reasoning:```It is clear that the "Cloudchaser Eagle" should be used to identify cards, and the status refers to the `status` column in the legalities table. However, what should be included in the output table and which column should be used to identify the "Cloudchaser Eagle" need further clarification.```
Example Ambiguity Type:```[‘AmbOutput’, ‘AmbColumn’’]```
Example Multiple Choice Clarification Question:```
What information should be included in the output table? a) the output table has one column with the unique statuses, b) the output table has one column with all statuses including duplicates, c) the output table has two columns with the unique statuses and the card name, d) other (please specify).
How to find the "Cloudchaser Eagle" cards? a) exact string match of "Cloudchaser Eagle" on the `name` column in the cards table, b) pattern string match of "%Cloudchaser Eagle%" on the `originalText` column in the cards table, c) exact string match of "Cloudchaser Eagle" on the `keywords` column in the cards table, d)other (please specify).
```
Example Question: ```For patients born between Year 1930 and 1940, how many percent of them were inpatient?```
Example Reasoning:```​​It is clear that the output table has a single column with the percentage information, and the where clause should include a filter based on the `birthday` column from the Patient table. However, finding the 'inpatient' and how to represent the percentage in the output table need to be further clarified.```
Example Ambiguity Type:```[‘AmbColumn’, ‘AmbValue’, ‘AmOutput’]```
Example Multiple Choice Clarification Question:```How to find the 'inpatient' information? a) 'inpatient' refers to having an exact match of '+' in the `Admission` column from the Patient table, b) inpatient refers to having a partial match of '%inpatient%' in the `Description` column from the Patient table, c) other (please specify).
How to format the percentage in the output table? a) the percentage is calculated as the number of inpatients divided by the number of patients times 100, b) the percentage is calculated as the number of inpatients divided by the number of patients, c) other (please specify).
```
'''

SRA = """Your task is to ask the user a single multiple choice clarification question to help you find the correct SQL query.
User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

QUESTION: ```{question}```
DATABASE SCHEMA:```{schema}```

Previous Clarification Questions and User Feedback: ```{cqs}```

List of Incorrect Queries: ```{sqls}```
The above SQL Queries are wrong.

Here are four ambiguity categories to consider:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be contained in the output table?
    - AmbValue: What predicate value should be used to filter results?

Let's think step by step.

STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions and incorrect queries.

STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to answer the QUESTION, considering each category individually.

STEP 3, Ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand.
        Use format: mul_choice_cq = "".
        
Answer:
"""

In [16]:
sql_generation = '''Complete sqlite SQL query only and with no explanation.
Your task is to write a SQLITE SQL QUERY that correctly answers the following QUESTION and align with database schema.

QUESTION: {question}

The DATABASE SCHEMA: {schema}

Answer:'''


sql_generation_v2 = '''Complete a SQLite SQL query only with no explanation.
Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

USER QUESTION: ```{question}```
The DATABASE SCHEMA: ```{schema}```

The user has provided a list of incorrect SQL queries and the following sql queries are wrong.
Incorrect Queries: ```[{sqls}]``` 

The user has provided some feedback. User Feedback are the golden truth.
User Feedback: {cqas}

You know the Incorrect Queries are wrong. Do not rewrite the exact same incorrect queries.
Please listen carefully to the user feedback. Write the new correct query.

Answer:
'''

fix_invalid_v1 = """Your task is to fix the invalid SQL query.

Database schema```{schema}```

Invalid SQL Query: ```{sql}```
The above SQL query is not valid and can not be executed.
Exception Message:```{ex}```

Fix the errors and output a new SQL query that is executable and align with Database schema.

Write the new executable SQL query in the format: sql = ``` ```.

Answer:
"""

In [17]:
sql_generation_selfdebug = '''Complete a SQLite SQL query only with no explanation.
Your task is to complete a new SQLite SQL QUERY that correctly answers the USER QUESTION, aligns with DATABASE SCHEMA, and follows USER FEEDBACK.

USER QUESTION: ```{question}```
The DATABASE SCHEMA: ```{schema}```

Incorrect Queries: ```[{sqls}]```
The above SQL predictions are wrong. Please fix the SQL.
Write the new correct query.

Answer:
'''


In [18]:
def startBenchBaseline(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)
        
        sql_prompt = sql_generation.format(schema=dbschema, question=nlq)
        sql_query, pscore= generation(sql_prompt)
        history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                   sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid SQL: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            sql_prompt = sql_generation_selfdebug.format(schema=dbschema, question=nlq,\
                                              sqls=",\n".join(query))
            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                     sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("After Fix Invalid SQL: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [23]:
def askClarificationQuestions(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source, baseline_history):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
        if index in baseline_history and baseline_history[index]["num_cq_asked"] == 0:
            history_log[index] = baseline_history[index].copy()
            continue
        
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)
        
        sql_prompt = sql_generation.format(schema=dbschema, question=nlq)
        sql_query, pscore= generation(sql_prompt)
        history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                 sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "Answer: "+cqs_and_answers[i]
                
            cq_prompt = SRA.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
            print("cq: ", cq, pscore)
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4_turbo_generation(feedback_prompt, stopWord="Let's think step by step:")
            print()
            print("feedback, ", feedback, pscore)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "User Feedback: "+cqs_and_answers[i] 
            sql_prompt = sql_generation_v2.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)

            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("Afrer Fix Invalid: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [32]:
# decide when to stop
SRA_ES = """Your task is to identify the remaining ambiguity.
If there are remaining ambiguities, then ask the user a single multiple choice clarification question to help you find the correct SQL query.
User does not understand SQL. The clarification question should be comprehensive by people with no coding experience.

QUESTION: ```{question}```
DATABASE SCHEMA:```{schema}```

Previous Clarification Questions and User Feedback: ```{cqs}```

List of Incorrect Queries: ```{sqls}```
The above SQL Queries are wrong.

Here are four ambiguity categories to consider:
    - AmbQuestion: Is the question itself ambiguous?
    - AmbTableColumn: Is there ambiguity in mapping the entities from QUESTION to tables and columns in DATABASE SCHEMA?
    - AmbOutput: What fields and how many fields should be contained in the output table?
    - AmbValue: What predicate value should be used to filter results?

Let's think step by step.

STEP 1, Summarize the information that is clear based on the ansewrs to previous clarification questions.

STEP 2, Evaluate whether AmbQuestion, AmbColumn, AmbOutput, and AmbValue remain in formulating a SQL query to correctly answer the QUESTION, considering each category individually.

STEP 3, If no remaining ambiguities are identified, then output "NO AMBIGUITY".
        Otherwise, ask a multiple-choice clarification question to clarify the remaining ambiguities and help you find the correct SQL query. Ensure it's simple enough for someone without coding experience to understand. Use format: mul_choice_cq = "".
        
Answer:
"""

In [33]:
def askCQsBreakNoAmb(data_frame, history_log, log_name, rounds, num_of_tests, model_name, data_source, baseline_history):
    assert model_name in ['gpt35turbo', 'gpt4turbo']
    generation = None
    if model_name == 'gpt35turbo':
        generation = GPT35_generation
    else:
        generation = GPT4_turbo_generation
    for index in range(num_of_tests):
        if index in history_log and "num_cq_asked" in history_log[index]:
            # skip tests already seen
            continue
            
        history_log[index] = {}
        history_log[index]['sql_log'] = []
        history_log[index]['cq_log'] = []
        history_log[index]['feedback_log'] = []
        order = 0
        d = data_frame.iloc[[index]] 
        gold = d['sql'].values[0]
        dbname = d['target_db'].values[0]
        nlq = d['nl'].values[0]
        dbschema = d['target_schema'].values[0]
        cqs_and_answers = []
        query = set()
        print("nl: ", nlq, index)
        print("gold: ", gold)
        sql_query = ''
        if index in baseline_history and 'sql_log' in baseline_history[index]:
            history_log[index]['sql_log'].append(baseline_history[index]['sql_log'][0])
            order, sql_prompt, sql_query, pscore = history_log[index]['sql_log'][0]
        else:
            sql_prompt = ''
            sql_prompt = sql_generation_few_shot_prompt.format(schema=dbschema, question=nlq)
            sql_prompt = fewshot_prefix + sql_prompt  
            sql_query, pscore= generation(sql_prompt)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
        order += 1
        sql_query = clean_query(sql_query)
        print("sql: ", sql_query, pscore)
        query.add(sql_query)
        execution, exception = evalfunc(sql_query, gold, dbname, data_source)
        
        if exception:
            most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
            query.remove(most_recent_sql)
            invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=d['nl'],\
                                                 sql=most_recent_sql, ex=exception[0])
            sql, pscore= generation(invalid_prompt)
            print("After Fix Invalid: ", sql, pscore)
            valid_sql = clean_query(sql.split("sql = ")[-1])
            history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
            order += 1
            query.add(valid_sql)
            execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
        if execution:
            history_log[index]['num_cq_asked'] = 0
            print()
            print("-----execution match-----")
            print()
            continue
        for turn in range(rounds):
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "Answer: "+cqs_and_answers[i]
                
            cq_prompt = SRA_ES.format(schema=dbschema, question=nlq,\
                                            sqls=",\n".join(query), cqs=cqas)
            cq_prompt = cq_prefix_v1 + cq_prompt
            cq, pscore= generation(cq_prompt)
            history_log[index]['cq_log'].append((order, cq_prompt, cq, pscore))
            order += 1
            print("cq: ", cq, pscore)
            if "NO AMBIGUITY" in cq:
                print()
                print("-----NO AMBGUITY-----")
                print()
                break
            if "mul_choice_cq = " in cq:
                cq = cq.split("mul_choice_cq = ")[-1]
            
            feedback_prompt = feedback_v2.format(query = gold, question = cq, nlq=nlq)
            feedback_prompt = feedback_prefix_v1 + feedback_prompt
            feedback, pscore= GPT4_turbo_generation(feedback_prompt, stopWord="Let's think step by step:")
            print()
            print("feedback, ", feedback, pscore)
            history_log[index]['feedback_log'].append((order, feedback_prompt, feedback, pscore))
            order += 1
            if "answer_to_cq =" in feedback:
                feedback = feedback.split("answer_to_cq =")[-1]
            cqs_and_answers.append(cq)
            cqs_and_answers.append(feedback)
        
            # fix incorrect sql based on user feedback
            cqas = ""
            for i in range(len(cqs_and_answers)):
                if i%2 == 0:
                    cqas += "Multiple Choice Clarification Question: "+cqs_and_answers[i]
                else:
                    cqas += "User Feedback: "+cqs_and_answers[i] 
            sql_prompt = sql_generation_v2.format(schema=dbschema, question=nlq,\
                                              sqls="\n".join(query), cqas=cqas)

            sql_query, pscore= generation(sql_prompt)
            print("sql: ", sql_query, pscore)
            history_log[index]['sql_log'].append((order, sql_prompt, sql_query, pscore))
            order += 1
            sql_query = clean_query(sql_query)
            query.add(sql_query)
            execution, exception = evalfunc(sql_query, gold, dbname, data_source)
            if exception:
                most_recent_sql = clean_query(history_log[index]['sql_log'][-1][2])
                query.remove(most_recent_sql)
                invalid_prompt = fix_invalid_v1.format(schema=dbschema, question=nlq,\
                                                       sql=most_recent_sql, ex=exception[0])
                sql, pscore= generation(invalid_prompt)
                print("Afrer Fix Invalid: ", sql, pscore)
                valid_sql = clean_query(sql.split("sql = ")[-1])
                query.add(valid_sql)
                history_log[index]['sql_log'].append((order, invalid_prompt, valid_sql, pscore))
                order += 1
                execution, _ = evalfunc(valid_sql, gold, dbname, data_source)
            if execution:
                history_log[index]['num_cq_asked'] = turn + 1
                print()
                print("********execution match*********")
                print()
                break
                
        if 'num_cq_asked' not in history_log[index]:
            history_log[index]['num_cq_asked'] = "Failed"
        print('')
        print("------next question------")
        print('')
    save(log_name, history_log)

In [37]:
# kaggel_break_early_histories_gpt4_turbo = {}
# askCQsBreakNoAmb(data_frame=df, history_log=kaggel_break_early_histories_gpt4_turbo,\
#                   log_name='./zeroshotlogs/kaggle_gpt4_turbo_break_cq_4round.pkl', rounds=4,\
#                   num_of_tests=208, model_name='gpt4turbo', data_source='kaggle',\
#                          baseline_history=kaggle_cq_histories_gpt4_turbo)

nl:  Name the most popular release on houston. 0
gold:  SELECT
  T2.groupName
FROM torrents AS T2
JOIN tags AS T1
  ON T1.id = T2.id
WHERE
  T1.tag = "houston"
ORDER BY
  totalSnatched DESC
LIMIT 1
sql:  
SELECT groupName FROM torrents WHERE artist = 'houston' ORDER BY totalSnatched DESC LIMIT 1
 1.0406509834849023
cq:  STEP 1: The clear information based on the question and database schema is that the query is looking for a "release" associated with "houston" that is the most popular. The popularity is measured by the "totalSnatched" column in the "torrents" table. The "artist" column is used to filter releases by "houston".

STEP 2: Evaluating ambiguities:

- AmbQuestion: The question is clear in asking for the most popular release associated with "houston".
- AmbTableColumn: There is no ambiguity in mapping entities from the question to tables and columns in the database schema. It's clear that the "torrents" table contains the necessary information.
- AmbOutput: The question does n

cq:  STEP 1:
Based on the answers to previous clarification questions, it is clear that:
- The most popular release should be identified by considering both releases by the artist named 'houston' and releases tagged with 'houston'.
- The popularity is determined by the total number of times a release has been snatched (totalSnatched).
- The output should include only the name of the release.
- The filtering should only consider the tag 'houston' for identifying releases, regardless of the artist.

STEP 2:
Evaluating Ambiguities:
- AmbQuestion: The question itself is now clear after the clarification that the most popular release is determined by the totalSnatched and considering releases tagged with 'houston'.
- AmbTableColumn: There might still be ambiguity in how to correctly join the torrents and tags tables to filter releases tagged with 'houston', considering the database schema provided.
- AmbOutput: The output is specified to include only the name of the release, so there is no 


feedback,  The clarification question is asking what information should be included for each of the top 5 most popular releases after 2000. To determine the correct answer, we need to look at the SELECT clause of the gold query.

The gold query is:
```sql
SELECT
  groupName
FROM torrents
WHERE
  groupYear > 2000
ORDER BY
  totalSnatched DESC
LIMIT 5
```

From the SELECT clause, we can see that the only information selected is `groupName`, which represents the name of the release. This directly corresponds to the information that will be displayed for each of the top 5 most popular releases.

Now, let's consider the correctness of each multiple choice answer based on the gold query:
- a) Just the name of the release (groupName) - This is correct as per the SELECT clause.
- b) The name of the release and the artist - This is incorrect because the artist is not selected in the query.
- c) The name of the release, the artist, and the year of release - This is incorrect because neither the

sql:  ```sql
SELECT AVG(CAST(weight AS INTEGER)) FROM player WHERE player_id IN (SELECT player_id FROM player_award WHERE award_id = 'TSN All-Star' AND notes LIKE '%3B%')
``` 1.0221355975094963
cq:  STEP 1:
The information that is clear based on the answers to previous clarification questions is that the 'third-base man' position in the TSN all-star award is specified using a specific abbreviation for third base, which is likely '3B'. This information helps to filter the players awarded for the third-base position in the TSN all-star award.

STEP 2:
- AmbQuestion: The question itself is clear; it asks for the average weight of players who were awarded as third-base men in the TSN all-star award.
- AmbTableColumn: There might still be ambiguity in mapping entities from the question to tables and columns in the database schema. Specifically, it's clear which tables to use ('player' and 'player_award'), but it's not explicitly stated how to join these tables correctly based on the databas

sql:  ```sql
-- Given the user feedback and the database schema provided, there is no direct mention of a "defense position" column in any of the tables. The user feedback suggests it is listed under a specific column in one of the tables provided, but without explicit column names related to defense positions in the schema, it's not possible to construct a correct query based on the information given. The user's feedback might be based on a misunderstanding or incorrect information since the schema does not list any table or column explicitly related to defense positions.
``` 1.4686417797446985
cq:  STEP 1: From the previous clarification question and user feedback, it is clear that the information about a player's defense position is believed to be listed under a specific column in one of the tables provided in the database schema. However, upon reviewing the database schema provided, there is no explicit mention of a column that directly indicates a player's defense position.

STEP 

...


feedback,  The gold query provided is: 
```sql
SELECT
  commod
FROM resultsdata15
WHERE
  concen > lod
```

Reasoning:
- The gold query selects only the 'commod' (commodity) from the resultsdata15 table where the concentration ('concen') is greater than the limit of detection ('lod'). 
- This indicates that the output table is intended to contain only the names of the commodities with pesticide concentrations higher than their limit of detection. 
- There is no mention or inclusion of commodity types ('commtype'), laboratory information, or any other additional details in the gold query.

Correctness of each multiple choice answer based on the gold query:
- a) Just the name of the commodities: This matches the output of the gold query, which only includes the commodity names.
- b) The name of the commodities and their types: The gold query does not include commodity types, so this option is incorrect.
- c) The name of the commodities, their types, and the laboratory that tested them: 

sql:  ```sql
SELECT s.variety, SUM(r.concen) AS total_pesticide_level
FROM sampledata15 s
JOIN resultsdata15 r ON s.sample_pk = r.sample_pk
WHERE s.commod = 'apple'
GROUP BY s.variety
ORDER BY total_pesticide_level DESC
LIMIT 1
``` 1.0028594948159075
cq:  STEP 1:
The information that is clear based on the answers to previous clarification questions is that the user is interested in identifying the top apple variety with the highest sum of pesticide levels. This specifies the method of aggregation (sum of pesticide levels) and indicates that the output should focus on a single apple variety, not a list or averages.

STEP 2:
- AmbQuestion: There is no ambiguity in the question itself after the clarification; it asks for the apple variety with the highest sum of pesticide levels.
- AmbTableColumn: It's clear that the `variety` column from the `sampledata15` table and the `concen` column from the `resultsdata15` table are relevant. However, it's not explicitly stated how to filter for "app

cq:  STEP 1: The clear information based on the question and database schema is that the user wants to know the average pesticide residue for each state in the US where food is grown. The database schema indicates that there are two tables, `sampledata15` and `resultsdata15`, which are linked by the `sample_pk` column. The `state` column in `sampledata15` can be used to identify the state, and the `concen` column in `resultsdata15` seems to represent the concentration of pesticide residue.

STEP 2: Evaluating ambiguities:

- AmbQuestion: There is no ambiguity in the question itself as it clearly asks for the average pesticide residue for each state.
- AmbTableColumn: It is clear which tables and columns to use for the query. The `state` column from `sampledata15` and the `concen` column from `resultsdata15` are relevant for the query.
- AmbOutput: The question specifies that the output should include the average pesticide residue for each state, implying that the output table should co

cq:  STEP 1: The clear information based on the question and database schema is that we are looking for the top 10 commodities with the highest residue in the year 2015. The database schema indicates that there are two tables, `sampledata15` and `resultsdata15`, which are linked by `sample_pk`. The `year` column in `sampledata15` can be used to filter samples from 2015, and the `concen` column in `resultsdata15` seems to represent the residue concentration.

STEP 2: Evaluating Ambiguities:

- AmbQuestion: The question is clear in its request for the top 10 commodities with the highest residue in 2015.
- AmbTableColumn: It's clear that `sampledata15.year` is used to filter for 2015, and `resultsdata15.concen` might be used to determine residue levels. However, it's not explicitly clear how "highest residue" is defined - is it the average, maximum, sum, or another measure of residue concentration?
- AmbOutput: The question specifies the top 10 commodities, but it does not clarify whether

sql:  ```sql
SELECT COUNT(*) FROM sampledata15 WHERE origin = '3'
``` 1.0000614878706369

********execution match*********


------next question------

nl:  What is the result in case 6B:E2:54:C6:58:D2? 206
gold:  SELECT
  Outcome
FROM GreaterManchesterCrime
WHERE
  CrimeID = "6B:E2:54:C6:58:D2"
sql:  
SELECT Outcome FROM GreaterManchesterCrime WHERE CrimeID = '6B:E2:54:C6:58:D2'
 1.0004519948562745

-----execution match-----

nl:  Which type of crime has the most investigation complete cases? 207
gold:  SELECT
  Type
FROM GreaterManchesterCrime
WHERE
  Outcome = "Investigation complete; no suspect identified"
GROUP BY
  Type
ORDER BY
  COUNT(*) DESC
LIMIT 1
sql:  
SELECT Type, COUNT(*) AS TotalCompleteCases 
FROM GreaterManchesterCrime 
WHERE Outcome = 'Investigation complete no suspect identified' 
GROUP BY Type 
ORDER BY TotalCompleteCases DESC 
LIMIT 1
 1.063624066418966
cq:  Given the information and the task at hand, let's proceed with the step-by-step evaluation:

### STEP 1: Su

In [50]:
kaggle_break_early_histories_gpt4_turbo = {}
with open('./zeroshotlogs/kaggle_gpt4_turbo_break_cq_4round.pkl', 'rb') as f:
    kaggle_break_early_histories_gpt4_turbo = pickle.load(f)
    
hist = collections.defaultdict(int)
for k, v in kaggle_break_early_histories_gpt4_turbo.items():
    if 'num_cq_asked' not in v:
        v['num_cq_asked'] = "Failed"
    hist[v['num_cq_asked']] += 1
print(hist)

defaultdict(<class 'int'>, {3: 5, 'Failed': 69, 1: 61, 0: 49, 2: 24})


In [39]:
# bird_break_early_histories_gpt4_turbo = {}
# askCQsBreakNoAmb(data_frame=bird_data_df, history_log=bird_break_early_histories_gpt4_turbo,\
#                   log_name='./zeroshotlogs/bird_gpt4_turbo_break_cq_4round.pkl', rounds=4,\
#                   num_of_tests=176, model_name='gpt4turbo', data_source='bird',\
#                          baseline_history=bird_cq_histories_gpt4_turbo)

nl:  Which different county has the most number of closed schools? Please provide the name of each school as well as the closure date. 0
gold:  SELECT DISTINCT County, School, ClosedDate FROM schools WHERE County = ( SELECT County FROM schools WHERE StatusType = 'Closed' GROUP BY County ORDER BY COUNT(School) DESC LIMIT 1 ) AND StatusType = 'Closed' AND school IS NOT NULL
sql:  
SELECT s.County, s.School, s.ClosedDate
FROM schools s
WHERE s.StatusType = 'Closed'
GROUP BY s.County
ORDER BY COUNT(s.School) DESC
LIMIT 1
 1.0568715604219951
cq:  Given the task and the database schema provided, let's evaluate the ambiguities step by step.

### STEP 1: Information Summary

- The question asks for the county with the most number of closed schools, including the name of each school and the closure date.
- The `schools` table contains relevant columns such as `StatusType` for identifying closed schools, `County` for the county name, `School` for the school name, and `ClosedDate` for the closure


feedback,  The portion of the gold query that answers the clarification question is the SELECT statement, specifically `SELECT T1.NumTstTakr`. This part of the query indicates what data is being retrieved from the database. Since there is no aggregation function (like SUM) applied to `NumTstTakr`, the query is set to return the number of test takers listed individually for each school whose mailing city address is in Fresno. There is no indication in the query that the numbers are summed up across all schools.

Given this analysis, the correct answer based on the gold query is:

answer_to_cq = "b) Listed individually for each school." 1.2629930824698912
sql:  ```sql
SELECT s.sname, ss.NumTstTakr
FROM schools s
JOIN satscores ss ON s.CDSCode = ss.cds
WHERE s.MailCity = 'Fresno'
``` 1.0340082892065463
no such column: s.sname
Afrer Fix Invalid:  sql = ```
SELECT ss.sname, ss.NumTstTakr
FROM schools s
JOIN satscores ss ON s.CDSCode = ss.cds
WHERE s.MailCity = 'Fresno'
``` 1.01581001032631


feedback,  The gold query specifically calculates the ratio of schools that are "Locally funded" compared to those that are not, within Santa Clara County charter schools. The query does this by summing up instances where `FundingType = 'Locally funded'` and comparing it to the sum of instances where `FundingType != 'Locally funded'`. This calculation inherently divides funding types into two categories: 'Locally funded' and all other types of funding without specifying what these other types include.

Now, let's evaluate the multiple-choice options based on the gold query:

a) 'Locally Funded' refers to schools funded by local government or community, and all other types include state, federal, or private funding. - This option seems to align with the query's division of funding types into 'Locally funded' and all others, but the query does not explicitly define what the other types include (state, federal, or private funding).

b) 'Locally Funded' includes any funding sourced within


feedback,  The portion of the gold query that answers the clarification question is: ```WHERE County = 'Santa Clara' AND Charter = 1```. This part of the query specifies that only charter schools within Santa Clara County are to be considered in the calculation. It filters the schools based on their location (Santa Clara County) and their charter status, indicating that schools outside of Santa Clara County should not be included in the comparison.

Now, considering the correctness of each multiple choice answer based on the gold query:

a) Yes, include all charter schools regardless of county. - This option is incorrect because the query specifically filters for schools within Santa Clara County.

b) No, only include charter schools within Santa Clara County. - This option is correct because the query specifies that the calculation should only consider charter schools within Santa Clara County.

c) Include only charter schools within Santa Clara County but consider all funding types 


feedback,  First, to address the portion of the clarification question regarding how non-chartered schools are identified, we look at the gold query. The query specifies `T2.Charter = 0` within the WHERE clause, indicating that the identification of non-chartered schools is based on the `Charter` column in the `schools` table having a specific value (in this case, 0). This directly answers the first part of the clarification question, making option b) the correct choice for identifying non-chartered schools.

Next, for determining the percent of eligible free meals for grades 1 through 12, the gold query does not use a column named `Percent (%) Eligible Free (K-12)`. Instead, it calculates the percentage by using `CAST(T1.`Free Meal Count (K-12)` AS REAL) * 100 / T1.`Enrollment (K-12)` < 0.18`. This calculation involves a combination of columns (`Free Meal Count (K-12)` and `Enrollment (K-12)`) to derive the percentage of eligible free meals, which is not directly mentioned in the opt

...

cq:  Given the information and the database schema provided, let's address the steps for identifying any remaining ambiguities in the question about transactions at gas stations in the Czech Republic with a price over 1000.

STEP 1: Summarize the clear information.
- The question asks for the number of transactions at gas stations in the Czech Republic where the transaction price is over 1000.
- The relevant tables for this query are `transactions_1k` for transaction details and `gasstations` for gas station locations.
- The condition involves filtering transactions based on the country ('Czech Republic') and a price threshold (over 1000).

STEP 2: Evaluate remaining ambiguities.
- AmbQuestion: The question itself is clear about what is being asked.
- AmbTableColumn: The mapping to `gasstations.Country` for the Czech Republic and `transactions_1k.Price` for the price condition seems straightforward. However, it's not specified whether the price threshold of 1000 refers to a specific cu


feedback,  The portion of the gold query that answers the clarification question is: ```INNER JOIN yearmonth AS T2 ON T1.CustomerID = T2.CustomerID WHERE T2.Date = '201309'```. This part of the query specifies that the consumption data is being retrieved from the 'yearmonth' table for September 2013, based on the 'Date' column matching '201309'.

Given this information, let's evaluate the multiple-choice answers:

a) Use the total consumption recorded in the 'yearmonth' table - This option directly corresponds to the method used in the gold query, where the 'yearmonth' table is used to calculate the total consumption for September 2013.

b) Calculate total consumption from all transactions in September 2013 in the 'transactions_1k' table - This option suggests using a different table ('transactions_1k') than the one used in the gold query ('yearmonth'). Therefore, this option does not match the method used in the gold query.

c) Consider another method (please specify) - Since the gol

cq:  Given the information and the task at hand, let's proceed with the step-by-step evaluation:

### STEP 1: Summarize the Clear Information

- The question asks for the segments (SME, LAM, KAM) that have the biggest and lowest percentage increases in consumption paid in EUR between 2012 and 2013.
- The database schema provided includes tables that could be relevant to answering this question, specifically the `customers` and `yearmonth` tables for segment information and consumption data, respectively.
- The currency filter for transactions in EUR is specified, indicating that the `Currency` column in the `customers` table is relevant.
- The time frame for comparison is clearly defined as between the years 2012 and 2013.

### STEP 2: Evaluate Remaining Ambiguities

#### AmbQuestion:
- The question itself is clear in what it asks for: the segments with the biggest and lowest percentage increases in consumption.

#### AmbTableColumn:
- There is no ambiguity in mapping the entities from

sql:  ```sql
WITH ConsumptionData AS (
  SELECT 
    c.Segment,
    SUM(CASE WHEN strftime('%Y', ym.Date) = '2012' THEN ym.Consumption ELSE 0 END) AS Consumption2012,
    SUM(CASE WHEN strftime('%Y', ym.Date) = '2013' THEN ym.Consumption ELSE 0 END) AS Consumption2013
  FROM 
    yearmonth ym
    JOIN customers c ON ym.CustomerID = c.CustomerID
  WHERE 
    c.Segment IN ('SME', 'LAM', 'KAM')
  GROUP BY 
    c.Segment
),
PercentageChange AS (
  SELECT 
    Segment,
    ((Consumption2013 - Consumption2012) / Consumption2012) * 100 AS PercentageIncrease
  FROM 
    ConsumptionData
),
MaxMinSegments AS (
  SELECT 
    Segment,
    PercentageIncrease,
    'Highest' AS ChangeType
  FROM 
    PercentageChange
  ORDER BY 
    PercentageIncrease DESC
  LIMIT 1
  UNION ALL
  SELECT 
    Segment,
    PercentageIncrease,
    'Lowest' AS ChangeType
  FROM 
    PercentageChange
  ORDER BY 
    PercentageIncrease
  LIMIT 1
)

SELECT 
  Segment,
  PercentageIncrease
FROM 
  MaxMinSegments
``` 1.041532

cq:  STEP 1: Summarize the information that is clear based on the answers to previous clarification questions.

- The user wants to see the names of the segments along with their respective percentage increases.
- The calculation of the percentage increase in consumption for each segment should include all consumption data for customers in the segments SME, LAM, and KAM, regardless of the currency of their transactions.

STEP 2: Evaluate remaining ambiguities.

- AmbQuestion: There is no ambiguity in the question itself after the clarification questions. It's clear that the user wants to know which of the three segments—SME, LAM, and KAM—has the biggest and lowest percentage increases in consumption paid in EUR between 2012 and 2013.
- AmbTableColumn: There might be ambiguity in how to accurately calculate consumption increases since the clarification questions have established that currency does not directly filter or specify transactions. However, it's not entirely clear if the consu

In [40]:
bird_break_early_histories_gpt4_turbo = {}
with open('./zeroshotlogs/bird_gpt4_turbo_break_cq_4round.pkl', 'rb') as f:
    bird_break_early_histories_gpt4_turbo = pickle.load(f)
    
hist = collections.defaultdict(int)
for k, v in bird_break_early_histories_gpt4_turbo.items():
    if 'num_cq_asked' not in v:
        v['num_cq_asked'] = "Failed"
    hist[v['num_cq_asked']] += 1
print(hist)

defaultdict(<class 'int'>, {'Failed': 93, 0: 58, 1: 20, 4: 1, 3: 3, 2: 1})
